<a href="https://colab.research.google.com/github/movi-data/painel_performance/blob/main/painel_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relatório Plataformas

In [3]:
import pandas as pd

# --- ETAPA 1: Carregar o arquivo ---
df_propostas = pd.read_csv("propostas.csv", sep=";", encoding="utf-8", low_memory=False)

# --- ETAPA 2: Preparar Colunas ---

# Tranformar 'Canal' em maiuscula
df_propostas['Canal'] = df_propostas['Canal'].str.upper()
# Filtrar o primeiro Canal na coluna 'Canal'
df_propostas['Canal'] = df_propostas['Canal'].str.split(',').str[0]

# --- ETAPA 3: Fazer agrupamento e ordernar ---

# Agrupar leads por mês na coluna 'Data' e 'Canal'
leads_por_canal_mes = df_propostas.groupby([pd.to_datetime(df_propostas["Data"], format="%d/%m/%Y", errors='coerce').dt.to_period("d"),
                                            "Canal"]).size().reset_index(name="Quantidade de Leads")




In [ ]:
# --- ETAPA 4: Criar arquivo csv e download ---

# Criar arquivo .csv
leads_por_canal_mes.to_csv("leads_por_canal_mes.csv",sep=";" ,index=False)

# Download do arquivo
from google.colab import files
files.download("leads_por_canal_mes.csv")

# Vizualização para conferência
leads_por_canal_mes.head()

In [ ]:
leads_por_canal_mes.tail()

,Data,Canal,Quantidade de Leads
1283,2025-09-16,CAMPANHA DIGITAL,1
388,2025-07-05,CAMPANHA DIGITAL,1
190,2025-06-18,AUTOCARRO,1
19,2025-06-02,USADOS BR,1
10,2025-06-02,CAMPANHA DIGITAL,1


# Completo

In [22]:
import pandas as pd

# --- ETAPA 1: Carregar os arquivos ---
df_propostas = pd.read_csv("propostas.csv", sep=";", encoding="utf-8", low_memory=False)
df_lojas = pd.read_excel("de_para_loja.xlsx")

# --- ETAPA : Preparar Colunas ---
# Tranformar 'Canal' em maiuscula
df_propostas['Canal'] = df_propostas['Canal'].str.upper()
# Filtrar o primeiro Canal na coluna 'Canal'
df_propostas['Canal'] = df_propostas['Canal'].str.split(',').str[0]

# --- ETAPA : Criar coluna 'Bot abandonado' ---
df_propostas['Bot Abandonado'] = df_propostas['tags'].str.contains('Bot Abandonado', case=False)

# --- ETAPA : #Adicionado nova coluna de acordo 'Origem'---

def tp_canal(Origem):
    if Origem == 'Internet':
        return 'Internet'
    elif Origem == 'MobiBot':
        return 'Internet'
    else:
      pass
df_propostas['TP Canal'] = df_propostas['Origem'].apply(tp_canal)

# --- ETAPA : Novo df com menos colunas ---
df_propostas_resumido = df_propostas[['ID Lead', 'Data', 'Data ultima interação',
                                      'Nome PDV','Origem', 'TP Canal', 'Canal',
                                       'tipo', 'qualificado',
                                       'vetor_id', 'Bot Abandonado'
                                      ]]

# --- ETAPA : Criar merge com base no Lojas ---
df_propostas_ajustado = pd.merge(df_propostas_resumido, df_lojas,
                                 on="Nome PDV", how="left", indicator=True)


# Substituir coluna 'Nome PDV' por 'Nome ajustado'
df_propostas_ajustado['Nome PDV'] = df_propostas_ajustado['Nome ajustado']

# Manter somente colunas necessárias
df_propostas_ajustado = df_propostas_ajustado[[ 'ID Lead', 'Data', 'Data ultima interação',
                                      'Nome PDV','Origem', 'TP Canal', 'Canal',
                                      'Status', 'tipo', 'qualificado',
                                       'vetor_id', 'Bot Abandonado', 'CNPJ',
                                      'Região','Varejo', 'UF',
                                      'Cidade', 'DDD',
                                      'Status', 'CEP', 'Telefone',
                                      'Endereço ', 'Bairro', 'Mobi ID',
                                       'Regional', '_merge'
                                        ]]

In [23]:
# --- ETAPA : Criar arquivo csv e download ---

# Criar arquivo .csv
df_propostas_ajustado.to_csv("propostas_ajustado.csv",sep=";" ,index=False)

# Download do arquivo
from google.colab import files
files.download("propostas_ajustado.csv")

# Vizualização para conferência
df_propostas_ajustado.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,ID Lead,Data,Data ultima interação,Nome PDV,Origem,TP Canal,Canal,Status,tipo,qualificado,...,Cidade,DDD,Status,CEP,Telefone,Endereço,Bairro,Mobi ID,Regional,_merge
0,55410293,01/06/2025,01/06/2025,Seminovos Movida Indaiatuba,MobiBot,Internet,WHATSAPP,Ativa,PERFORMANCE,False,...,Indaiatuba,19,Ativa,13338-705,19 3121 9005,Avenida Presidente Vargas 2939 Vila Vitória,Vila Homero,60901,Leonardo Alves Cardoso,both
1,55410294,01/06/2025,17/06/2025,Seminovos Movida Belo Horizonte Raja,MobiBot,Internet,WHATSAPP,Ativa,PERFORMANCE,True,...,Belo Horizonte,31,Ativa,30494-310,31 30582177,Avenida Raja Gabaglia 3500 LOJA 1,Estoril,18983,Ari Gomes De Oliveira,both
2,55410295,01/06/2025,01/06/2025,Seminovos Movida Várzea Grande,MobiBot,Internet,WHATSAPP,Ativa,PERFORMANCE,False,...,Cuiabá,65,Ativa,78115-005,65 40420514,Avenida da Feb 1788,Da Manga,18994,Kaique Silva Gorayeb,both
3,55410296,01/06/2025,01/06/2025,Seminovos Movida Recife Espinheiro,MobiBot,Internet,WHATSAPP,Ativa,PERFORMANCE,False,...,Recife,81,Ativa,52021-010,81 20113462,Rua Alfredo de Castro 101,Espinheiro,51076,Wellington Valfrido De Santana,both
4,55410297,01/06/2025,01/06/2025,Seminovos Movida João Pessoa Brisamar,MobiBot,Internet,WHATSAPP,Ativa,PERFORMANCE,False,...,João Pessoa,83,Ativa,58033-454,83 31420347,Rua Empresário Clóvis Rolim 2001 Brisamar,Brisamar,58919,Wellington Valfrido De Santana,both


In [24]:
print(df_propostas_ajustado.columns)

Index(['ID Lead', 'Data', 'Data ultima interação', 'Nome PDV', 'Origem',
       'TP Canal', 'Canal', 'Status', 'tipo', 'qualificado', 'vetor_id',
       'Bot Abandonado', 'CNPJ', 'Região', 'Varejo', 'UF', 'Cidade', 'DDD',
       'Status', 'CEP', 'Telefone', 'Endereço ', 'Bairro', 'Mobi ID',
       'Regional', '_merge'],
      dtype='object')


In [25]:
df_propostas_ajustado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388306 entries, 0 to 388305
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   ID Lead                388306 non-null  int64   
 1   Data                   388306 non-null  object  
 2   Data ultima interação  388306 non-null  object  
 3   Nome PDV               388306 non-null  object  
 4   Origem                 388306 non-null  object  
 5   TP Canal               328939 non-null  object  
 6   Canal                  373529 non-null  object  
 7   Status                 387695 non-null  object  
 8   tipo                   388306 non-null  object  
 9   qualificado            388306 non-null  bool    
 10  vetor_id               7011 non-null    float64 
 11  Bot Abandonado         233050 non-null  object  
 12  CNPJ                   386120 non-null  object  
 13  Região                 387695 non-null  object  
 14  Varejo              

In [15]:
df_propostas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388306 entries, 0 to 388305
Data columns (total 38 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ID Lead                           388306 non-null  int64  
 1   Data                              388306 non-null  object 
 2   Data ultima interação             388306 non-null  object 
 3   Nome PDV                          388306 non-null  object 
 4   CNPJ PDV                          388306 non-null  int64  
 5   Nome Cliente                      384902 non-null  object 
 6   Email Cliente                     241314 non-null  object 
 7   Telefone Cliente                  388052 non-null  object 
 8   CPF                               10725 non-null   float64
 9   Lead                              51381 non-null   object 
 10  Origem                            388306 non-null  object 
 11  Canal                             373529 non-null  o

In [16]:
df_propostas.columns

Index(['ID Lead', 'Data', 'Data ultima interação', 'Nome PDV', 'CNPJ PDV',
       'Nome Cliente', 'Email Cliente', 'Telefone Cliente', 'CPF', 'Lead',
       'Origem', 'Canal', 'Status', 'CallCenter status', 'Data Status Loja',
       'Nome Vendedor', 'ID do Vendedor', 'Callcenter detalhe',
       'callcenter status detalhe', 'assistente vendas',
       'ID do Assistente de Vendas', 'tipo', 'qualificado', 'documentos',
       'tags', 'vetor_id', 'data_ult_solicitacao_agendamento',
       'data_ult_agendamento', 'agendamento_visita', 'atraso_ult_agendamento',
       'qtd_agendamentos_90d', 'qtd_nao_foi_90d', 'regiao', 'Cidade', 'UF',
       'CEP', 'campanha', 'Bot Abandonado'],
      dtype='object')

In [ ]:
# --- ETAPA : Criar coluna de data ---

# Converter a coluna Data para datetime
df_propostas_ajustado["Data"] = pd.to_datetime(df_propostas_ajustado["Data"], format="%d/%m/%Y")